In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy
from scipy.stats import norm

In [2]:
#read data from "MacroSeries.txt"

macro_data = pd.read_csv("MacroSeries.txt", header = None)
macro_data.columns = ["c", "k", "w", "r", "y"]
c_vec = np.array(macro_data["c"])
k_vec = np.array(macro_data["k"])
w_vec = np.array(macro_data["w"])
r_vec = np.array(macro_data["r"])
y_vec = np.array(macro_data["y"])



The model is given by 
$$c_t^{-1} - \beta \mathbb{E}[r_{t+1} (c_{t+1})^{-1}] = 0$$
$$c_t + k_{t+1} - w_t - r_t k_t = 0$$
$$w_t - (1-\alpha)e^{z_t} (k_t)^\alpha = 0$$
$$r_t - \alpha e^{z_t} (k_t) ^{\alpha - 1} = 0$$
$$z_t = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t $$
$$y_t = e ^{z_t} (k_t)^\alpha $$



# (a) Estimate $\alpha, \beta, \rho, \mu$ by GMM

$$E[\epsilon_t] = 0$$


In [34]:
# given a guess of the parameters, use eq4 to get a sequence of z_t

# need initial guess for alpha
def get_z(r_vec, k_vec, alpha):
    return np.log(r_vec) - np.log(alpha) - (alpha -1)* np.log(k_vec)

#model moments are 0,0,0,0
#calculate the four data moments 
def data_moments(c_vec, k_vec, w_vec, r_vec, y_vec, param_guess):
    alpha, beta, rho, mu = param_guess
    z_vec = get_z(r_vec, k_vec, alpha)
    eq1 = z_vec[1:] - rho*z_vec[:-1] - (1-rho)*mu
    eq2 = (z_vec[1:] - rho*z_vec[:-1] - (1-rho)*mu) * z_vec[:-1]
    eq3 = beta * alpha * np.exp(z_vec[1:]) * (k_vec[1:]**(alpha-1)) * c_vec[:-1] / c_vec[1:] - 1
    eq4 = (beta * alpha * np.exp(z_vec[1:]) * (k_vec[1:]**(alpha-1)) * c_vec[:-1] / c_vec[1:] - 1) * w_vec[:-1]
    return np.array([eq1.mean(), eq2.mean(), eq3.mean(), eq4.mean()])


#since data moments would approach zero, it is better to just use the simple error vector
def error_vec(data_moments, model_moments):
    return (data_moments - model_moments)

def crit(param_vec, args):
    c_vec, k_vec, w_vec, r_vec, y_vec, W = args
    d_m = data_moments(c_vec, k_vec, w_vec, r_vec, y_vec, param_vec)
    m_m = np.zeros(4)
    err = error_vec(d_m, m_m)
    
    return err.T @ W @ err

    
    
bnds = ((1e-5, 1-1e-5), (1e-5, 1-1e-5), (-1+1e-5, 1-1e-5), (1e-5, None))
W = np.eye(4)
param_guess = np.array([0.4, 0.9, 0.7, 9])
argum = np.array([c_vec, k_vec, w_vec, r_vec, y_vec, W])

soln = opt.minimize(crit, param_guess, args = argum, bounds = bnds, options={'eps': 1e-04})
soln

      fun: 216335.73629772136
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.56305308e+02,  3.65903054e+07, -3.62094629e+01, -1.32172526e+01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 165
      nit: 16
   status: 0
  success: True
        x: array([0.42854715, 0.98995021, 0.70616479, 9.00139022])